In [2]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel("./data/cda1_1.xlsx")
data.head(5)

,主页ID,主页名称,分类,message,name,description,发文类型,y
0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,359000000000,公视粉丝团,电视频道,您所选择的电视频道，将会决定您的未来~\n遥控器的决定权，就在你手上！\n\n#看见更好的未来,2015公视电视募款 公视让你看见更好的未来,这是一个资讯爆炸的时代，你所选择的资讯将决定你的视野；你所选择的电视频道，将会决定你的未来 ...,video,L
2,359000000000,公视粉丝团,电视频道,好演员值得更多肯定和掌声！\n\n吴慷仁提到大概有六、七年资历的演员，所面临的最大困境，是赚...,公视粉丝团,NaN,link,L
3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
4,359000000000,公视粉丝团,电视频道,「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！」\n\n「两...,【我们的岛】抢救鹿港,叶明兰、叶镇中 / 采访报导 「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不...,link,L


In [4]:
grp = data.pivot_table(index='主页ID', columns='y', values=['分类'], aggfunc='count').droplevel(0, axis=1)
grp = grp.fillna(0)
blacklist = grp[ ((grp['H'] <= 1) & (grp['L'] >5)) ]
whitelist = grp[ (grp['L'] == 0) & (grp['H'] > 1) ]
data_remove_bw = data[ (~data['主页ID'].isin(blacklist.index.values)) &
                       (~data['主页ID'].isin(whitelist.index.values)) ]
data_remove_bw = data_remove_bw.reset_index()

In [5]:
counts50 = grp[(grp['H'] + grp['L'])>=50]
counts50['hl_score'] = np.log2(counts50['H'] + 0.1) / np.log2(counts50['H'] + counts50['L'] + 0.1)
data_tmp = pd.merge(data_remove_bw, counts50, how='left', on='主页ID')
data_hl_score = data_tmp['hl_score'].fillna(0)
data_hl_score

<ipython-input-5-2e605e0eb0d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts50['hl_score'] = np.log2(counts50['H'] + 0.1) / np.log2(counts50['H'] + counts50['L'] + 0.1)


0       0.000000
1       0.000000
2       0.767869
3       0.000000
4       0.734113
          ...   
6483    0.372763
6484    0.398857
6485    0.398857
6486    0.767869
6487    0.261170
Name: hl_score, Length: 6488, dtype: float64

In [6]:
from Python.pandas.feature_selection import *
fl_mapping = {'电视频道': 0, '网路社群': 1, '新闻': 2, '书籍杂志': 3, '电视剧': 4, '电影公司': 5, " \
             "'节目': 6, '媒体其他': 7, '电影': 5, '音乐': 7, '电影相关活动': 5, '广播': 7}
data_fl, fl_ohe = one_hot_encoder_column(data_remove_bw, '分类', fl_mapping, fill_na=7)
data_fl = pd.DataFrame(data_fl)
data_fl

,分类_0,分类_1,分类_2,分类_3,分类_4,分类_5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
6483,0.0,0.0,0.0,0.0,0.0,0.0
6484,0.0,0.0,0.0,1.0,0.0,0.0
6485,0.0,0.0,0.0,1.0,0.0,0.0
6486,0.0,1.0,0.0,0.0,0.0,0.0


In [7]:
doc_type_map = {'link': 1, 'photo': 2, 'video': 3}
data_doc_type, ohe_doc_type = one_hot_encoder_column(data_remove_bw, '发文类型', doc_type_map, fill_na=3)
data_doc_type

,发文类型_0,发文类型_1
0,0.0,1.0
1,1.0,0.0
2,0.0,0.0
3,1.0,0.0
4,0.0,0.0
...,...,...
6483,0.0,0.0
6484,1.0,0.0
6485,1.0,0.0
6486,0.0,0.0


In [8]:
from Python.nlp.nlp import *
stop_words = get_stop_words('./data/stop_words.txt', encoding='utf-8')
data_all = data.apply(lambda x: split_word(
    str(x['message']) + str(x['name']) + str(x['description']),
    stop_words, ['蔡依林', '叶明兰','叶镇中', '宋家豪', '白先勇','林智胜']),
                      axis=1)

data_all.to_csv("data/lines.csv")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\proffl\AppData\Local\Temp\jieba.cache


total counts:  1913


Loading model cost 0.977 seconds.
Prefix dict has been built successfully.


In [9]:
data_nosplit = data.apply(lambda x: str(x['message']) + str(x['name']) + str(x['description']), axis=1)
data_nosplit.to_csv("data/no_lines.csv")

In [27]:
from ltp import LTP
ltp = LTP(path="C:/Users/proffl/Downloads/small")

KeyError: 'transformer_config'

In [11]:
seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
pos = ltp.pos(hidden) # 词性标注
ner = ltp.ner(hidden) # 命名实体识别
srl = ltp.srl(hidden) # 语义角色标注
dep = ltp.dep(hidden) # 依存句法分析
sdp = ltp.sdp(hidden) # 语义依存分析(树)
# sdp = ltp.sdp(hidden, mode='graph')  # 语义依存分析(图)

In [12]:
data_nosplit = data.apply(lambda x: str(x['message']) + str(x['name']) + str(x['description']), axis=1)

In [20]:
data_nosplit = data_nosplit.tolist()
data_nosplit

['中华队长林智胜 三分砲！！！\n人如其名\u3000真的「致胜」阿～nannan',
 '您所选择的电视频道，将会决定您的未来~\n遥控器的决定权，就在你手上！\n\n#看见更好的未来2015公视电视募款 公视让你看见更好的未来这是一个资讯爆炸的时代，你所选择的资讯将决定你的视野；你所选择的电视频道，将会决定你的未来 2015公视电视募款http://www.pts.org.tw/donate2015_end/donate.html 公视的捐款专线：02-2633-9922；邮政划拨帐号：19213335，户名：?公共电视 另外您也可以...',
 '好演员值得更多肯定和掌声！\n\n吴慷仁提到大概有六、七年资历的演员，所面临的最大困境，是赚不到钱。但去年这时，曾有中国大陆的演出机会找他，「报酬很高，一个念头，就是几百万和几万的差别，我想了很久，如果去了，也许三个月就可以赚到几百万，但是我后来又想，在我这个年纪，做有意义的事情还是很重要的」。\n\n#影帝的思考就是帅 #麻醉风暴 #一把青 #吴慷仁 吴慷仁Kang Ren Wu公视粉丝团nan',
 '大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！来自 Mobile01 贴文的相片nan',
 '「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！」\n\n「两百年以上的老房子，消失的速度非常快，大概每个月，就有一栋被拆！」\n\n一府二鹿三艋舺，曾经是清朝重要商港的鹿港小镇，因为保留老城区的风貌，每年湧入六百万以上的游客，但是，随著商机进入，一间间老房子，开始被拆除改建。当一间又一间百年历史的老屋消失，鹿港还是鹿港吗？\n\n我们的岛-- 抢救鹿港  今晚十点播出【我们的岛】抢救鹿港叶明兰、叶镇中 / 采访报导 「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！「两百年以上的老房子，消失的速度非常快，大概每个月，就有一栋被拆！」 一府二鹿三艋舺，曾经是清朝重要商港的鹿港小镇，因为保留老城区的风貌，每年湧入六百万以上的游客，但是，随著商机进入，一间间老房子，开始被拆除改建。当一间又一间百年历史的老屋消失，鹿港还是鹿港吗？ 有一…',
 '爽啦～～～（ Supermm）\n\n#中华队 #古巴\nhttp://www.ettoday.net/news/201511

In [28]:
seg, hidden = ltp.seg(data_nosplit[:10])
ner = ltp.ner(hidden)
ner


[[('Ni', 0, 0), ('Nh', 1, 1)],
 [],
 [('Nh', 9, 9), ('Ns', 37, 37), ('Nh', 102, 102)],
 [],
 [('Ns', 20, 20),
  ('Ns', 50, 50),
  ('Ns', 53, 53),
  ('Ns', 85, 85),
  ('Ns', 86, 86),
  ('Ns', 97, 97),
  ('Ns', 99, 99),
  ('Ns', 119, 119),
  ('Ns', 144, 144)],
 [('Ni', 3, 3),
  ('Ns', 5, 5),
  ('Nh', 10, 10),
  ('Ns', 17, 17),
  ('Ni', 22, 22),
  ('Nh', 23, 23),
  ('Ns', 53, 53),
  ('Ns', 65, 65)],
 [('Ns', 4, 4), ('Ni', 13, 13)],
 [('Nh', 13, 13), ('Nh', 20, 20), ('Ni', 24, 24), ('Nh', 34, 34)],
 [('Nh', 9, 9), ('Nh', 18, 18)],
 [('Ni', 16, 16),
  ('Ns', 18, 18),
  ('Ni', 33, 33),
  ('Ns', 36, 36),
  ('Ni', 37, 37),
  ('Ni', 53, 53)]]